In [ ]:
from os.path import join as pjoin
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from tqdm import tqdm

import tensorflow as tf
import tensorflow_addons as tfa
tfkl = tf.keras.layers
from tqdm.keras import TqdmCallback
import json
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
import time
import math
import os

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

from preprocess_models.full_length import Preprocess
from models.LSTM import get_model
from utils import *

from sklearn.metrics import multilabel_confusion_matrix

import imageio
from PIL import Image

# Training loop gru

In [ ]:
from utils import load_relevant_data_subset
# from preprocess_models.interpolated_ragged import Preprocess
from preprocess_models.interpolated_notragged import Preprocess
from datasets.ragged_sym import get_KFold_dataset
from models.gru import get_model

folds = 7
path = "preprocessed_datasets/interpolated_notragged_nanfinalmask/"

ds = get_KFold_dataset(Preprocess(10), (None, 122), path)
hp = {
    "gru1": 96,
    'nhead': 12,
    'ff_dim': 160,
    'input_dropout': 0.3,
    'gru2': 96,
    'output_dropout': 0.2
}
hp = {
    "gru1": 80,
    'nhead': 16,
    'ff_dim': 192,
    'input_dropout': 0.2,
    'gru2': 128,
    'output_dropout': 0.3
}

get_model(hp).summary()

In [ ]:
val_accs = list()  
for fold_idx in range(folds):
    start = time.time()
    print(f'\n            *** Fold {fold_idx} ***\n')
    train_ds = ds.filter(lambda v, l, g, pid, sid: g != fold_idx).map(detuple).padded_batch(32)
    valid_ds = ds.filter(lambda v, l, g, pid, sid: g == fold_idx).map(detuple).padded_batch(32)
    model = get_model(hp)

    lr = 1e-3

    model.compile(
        tf.keras.optimizers.Adam(learning_rate=lr),
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            tf.keras.metrics.SparseCategoricalAccuracy(),
        ]
    )
    
    hist = model.fit(
        x=train_ds,
        epochs=50,
        verbose=2,
        callbacks=[
            TqdmCallback(verbose=0),
            # TimeLimitCallback(start, 1, 10), 
            tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.ModelCheckpoint(pjoin(path.split('/')[0], f"model_{fold_idx}"), 
                save_best_only=True, 
                save_weights_only=True,
                restore_best_weights=True, 
                monitor="val_sparse_categorical_accuracy", mode="max"),
            tf.keras.callbacks.EarlyStopping(patience=10, monitor="val_sparse_categorical_accuracy", mode="max", restore_best_weights=True)
            ],
        validation_data=valid_ds,
        validation_freq=1,
        workers=2,
        use_multiprocessing=True
    )

    best_acc = max(hist.history['val_sparse_categorical_accuracy'])
    print("Best acc fold", fold_idx, ":\n ->", 100*round(best_acc, 4), "%")
    val_accs.append(
        best_acc
    )
    break

print("Bagged final valid acc score:")
bagged_score = 100*np.round(np.array(val_accs).mean(), 4)
print(bagged_score, "%")

globalement 50 s pour une epoch -> voir effet batch size réduit sur généralisation ?

Epoch 47/100

2520/2520 - 52s - loss: 0.5897 - sparse_categorical_accuracy: 0.8558 - val_loss: 1.9944 - val_sparse_categorical_accuracy: 0.5938 - lr: 2.0242e-05 - 52s/epoch - 21ms/step

# Training loop ViViT

We will perform both modal ("spatial") and temporal attention

Get data

In [ ]:
from preprocess_models.multimodal_vivit import Preprocess
from datasets.vivit import get_KFold_dataset

path = "preprocessed_datasets/vivit_experiment_centerpad_50ts/"
ds = get_KFold_dataset(Preprocess(50), path)

from models.vivitlike import get_model
model = get_model({
    'emb_dropout':0.,
    'dim_model':256,#177
    'num_enc':1,
    # 'ff_dim':1024,
    'nhead':8,
    't_dropout':0.4
})

In [ ]:
#example to output a preprocessed gif

BASE_DIR = "asl-signs"
df = pd.read_csv(pjoin(BASE_DIR, "train.csv"))
df['path'] = "/Users/gus/Desktop/envs/asl/asl-signs/"+df['path']

#random path 
sample = df.sample(1)
pq_path = sample.path.iloc[0]
print(sample.sign.iloc[0])
from utils import *
lm = load_relevant_data_subset(pq_path)

p = Preprocess(64)
p.output_gif_result(lm)
# p.output_gif_result(right_handed)

In [ ]:
fold_idx = 0
train_ds = ds.filter(lambda X, y: X['group'] != fold_idx).shuffle(int(1e5)).batch(32)
valid_ds = ds.filter(lambda X, y: X['group'] == fold_idx).batch(256)

with tf.device('/cpu:0'):
    lr = 1e-3

    model.compile(
        tf.keras.optimizers.legacy.Adam(learning_rate=lr),
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            tf.keras.metrics.SparseCategoricalAccuracy(),
        ]
    )

    hist = model.fit(
        x=train_ds,
        epochs=100,
        verbose=1,
        callbacks=[
            tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.ModelCheckpoint(pjoin(path.split('/')[0], f"model_{fold_idx}"), 
                save_best_only=True, 
                save_weights_only=True,
                restore_best_weights=True, 
                monitor="val_sparse_categorical_accuracy", mode="max"),
            tf.keras.callbacks.EarlyStopping(patience=10, monitor="val_sparse_categorical_accuracy", mode="max", restore_best_weights=True)
            ],
        validation_data=valid_ds,
        validation_freq=1,
        workers=2,
        use_multiprocessing=True
    )